# <div align="center" > The custom environment for the Project Portfolio simulation </div>
<div align="center" > This is the simulation notebook for the project portfolio environement simulation.
</br>
</br>


# 1. Introduction
<span style="padding-left: 2em;">
This continues the paragraph normally.here we're going to elaborate on the problem definition and design choices.</span>

## 1.1. Overview on Project Portfolio Management
<span style="padding-left: 2em;">
This continues the paragraph normally.here we're going to elaborate on the problem definition and design choices.</span>

### 1.1.1. Project Cost Performance Baseline
<span style="padding-left: 2em;">
This is the main diagram to evaluate and analize porfolio budgeting status and performance</span>

<img style="
padding-left: 2em; 
margin: auto;
display: block;
" src="../Assets/Project-cost-baseline-graph.jpg" alt="drawing" width="400"/>


## 1.2. Deffinition of the Problem
<span style="padding-left: 2em;">
This continues the paragraph normally.here we're going to elaborate on the problem definition and design choices.</span>



## 1.3. environment design 
<span style="padding-left: 2em;">
The desing for the environment consists of several key components.</span>

<span style="padding-left: 2em;">
here's an infograph of environment and it's architecture</span>

<img style="
padding-left: 2em; 
margin: auto;
display: block;
" src="../assets/Info_graphs/environment/Mindmap - 2025.10.30-Environment.svg" alt="drawing" width="400"/>

## 1.4. Considerations and side notes
<span style="padding-left: 2em;">
1. Standard Macro economical parameters:</span>
<br/>
<span style="padding-left: 4em;">
1.1. 4% anual inflation on costs</span>

# 🚀Initiation

In [1]:
from gymnasium import Env
from gymnasium.spaces import Discrete, Box
import numpy as np


In [2]:
import torch
print(torch.__version__)
print("CUDA available:", torch.cuda.is_available())
print("Device name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "CPU only")

2.0.1+cu117
CUDA available: True
Device name: NVIDIA GeForce 940MX
